# Loading code

In [84]:
import pdfplumber
import pandas as pd

tables = []

with pdfplumber.open("mvfodevarer.pdf") as pdf:
    # Iterate through each page
    for page in pdf.pages:
        # Extract tables from the page
        page_tables = page.extract_tables({
           'vertical_strategy': 'lines',
           'horizontal_strategy': 'lines',
           'intersection_x_tolerance': 8,
           'intersection_y_tolerance': 8
        })
 

        if page_tables:
            for table in page_tables:
                if table:
                    tables.append({
                        'page': pdf.pages.index(page) + 1,
                        'data': table
                    })


# for i, item in enumerate(tables[0]):
#     #Detect new 
#     #
#     #
#     #
#     #

In [85]:
len(tables)

38

In [86]:


# for table in tables:
#     for row in table['data']:    
#         if row.__contains__('Madvare'):
#             print(f"Page: {table['page']}")
#             print(row)
        
tables[0]['data'] 

[['', '', None, None, None, None, None, None, None, None, None, ''],
 ['', 'Brød', None, None, None, None, None, None, None, None, None, ''],
 ['', '', None, '', None, None, '', None, None, '', None, ''],
 ['', '', '', '', '', '', '', '', '', '', '', ''],
 ['', 'Madvare', '', '', 'Lille', '', '', 'Mellem', '', '', 'Stor', ''],
 ['', 'Bagel', '', '', '', '', '', '90', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', '', ''],
 ['',
  'Bolle, almindelig / fuldkorn – bager/industri',
  '',
  '',
  '40',
  '',
  '',
  '60',
  '',
  '',
  '80',
  ''],
 ['',
  'Bolle, almindelig / fuldkorn – hjemmebagt',
  '',
  '',
  '50',
  '',
  '',
  '70',
  '',
  '',
  '120',
  ''],
 ['', '', '', '', '', '', '', '', '', '', '', ''],
 ['',
  'Burger/hamburger bolle, almindelig/fuldkorn',
  '',
  '',
  '',
  '',
  '',
  '60',
  '',
  '',
  '90',
  ''],
 ['',
  'Ciabattabolle, almindelig/fuldkorn',
  '',
  '',
  '',
  '',
  '',
  '70',
  '',
  '',
  '',
  ''],
 ['', '', '', '', '', '', '', '', '',

In [177]:
import tabulate
df = pd.DataFrame(tables[0]['data'])
print(df.to_markdown())

|    | 0   | 1                                             | 2   | 3   | 4       | 5   | 6   | 7        | 8   | 9   | 10      | 11   |
|---:|:----|:----------------------------------------------|:----|:----|:--------|:----|:----|:---------|:----|:----|:--------|:-----|
|  0 |     |                                               |     |     |         |     |     |          |     |     |         |      |
|  1 |     | Brød                                          |     |     |         |     |     |          |     |     |         |      |
|  2 |     |                                               |     |     |         |     |     |          |     |     |         |      |
|  3 |     |                                               |     |     |         |     |     |          |     |     |         |      |
|  4 |     | Madvare                                       |     |     | Lille   |     |     | Mellem   |     |     | Stor    |      |
|  5 |     | Bagel                                     

# Extraction code

### Remove empty colums and rows

In [178]:
df.replace("", float("NaN"), inplace=True)
df.dropna(how='all', axis=1, inplace=True) #Columns
df.dropna(how = 'all', axis = 0,  inplace = True) #Row
print(df.to_markdown())

|    | 1                                             | 4       | 7        | 10      |
|---:|:----------------------------------------------|:--------|:---------|:--------|
|  1 | Brød                                          |         |          |         |
|  4 | Madvare                                       | Lille   | Mellem   | Stor    |
|  5 | Bagel                                         | nan     | 90       | nan     |
|  7 | Bolle, almindelig / fuldkorn – bager/industri | 40      | 60       | 80      |
|  8 | Bolle, almindelig / fuldkorn – hjemmebagt     | 50      | 70       | 120     |
| 10 | Burger/hamburger bolle, almindelig/fuldkorn   | nan     | 60       | 90      |
| 11 | Ciabattabolle, almindelig/fuldkorn            | nan     | 70       | nan     |
| 13 | Crouton                                       | 0,4     | nan      | nan     |
| 14 | Flute/baguette – langt                        | nan     | 150      | 300     |
| 16 | Flute/baguette – mini/kuvert                  |

/var/folders/95/6_45t3ps6xx37f6psgqx50900000gq/T/ipykernel_28216/378557771.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("", float("NaN"), inplace=True)


### Remove everything above headers

In [179]:
#Find index where "Madvare"-string is present
matching_rows = df[df.apply(lambda row: row.astype(str).str.contains('Madvare', case=False, na=False).any(), axis=1)]

# Get the indices of the matching rows
idx = matching_rows.index.tolist()[0]
print(f"idx: {idx}")
#Extract the cells of this row as these are going to function as header
new_columns = df.loc[int(idx)].to_list()
df.columns = new_columns

#Slice away from "Madvare"-row to the top
df = df.loc[idx+1:]

#df = df[int(idx+1):]
# print(df.to_markdown())

idx: 4


In [180]:
df

,Madvare,Lille,Mellem,Stor
5,Bagel,NaN,90,NaN
7,"Bolle, almindelig / fuldkorn – bager/industri",40,60,80
8,"Bolle, almindelig / fuldkorn – hjemmebagt",50,70,120
10,"Burger/hamburger bolle, almindelig/fuldkorn",NaN,60,90
11,"Ciabattabolle, almindelig/fuldkorn",NaN,70,NaN
13,Crouton,"0,4",NaN,NaN
14,Flute/baguette – langt,NaN,150,300
16,Flute/baguette – mini/kuvert,NaN,50,NaN
17,Focacciabolle,NaN,120,NaN
19,"Hvedebrød, skive industribrød",1 cm=25,1½ cm=30,2cm=40
